## plot time series of hindcast and forecast at lead year ii

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import numpy as np
from scipy.stats import pearsonr
from matplotlib import cm,colors,gridspec
import scipy.signal as sgl
import os

In [2]:
USER = os.environ['USER']

## FOSI hindcast 

In [3]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{datadir}/DPLE-FOSI_hindcast_Phi_LME_1948-2017_yearly.nc'
filename = f'{datadir}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_yearly.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:          (lme: 11, year: 70)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
    time             (year) int64 ...
Dimensions without coordinates: year
Data variables: (12/26)
    Phi_0_0-200m     (lme, year) float64 ...
    Phi_0_200-600m   (lme, year) float64 ...
    Phi_1_0-200m     (lme, year) float64 ...
    Phi_1_200-600m   (lme, year) float64 ...
    Phi_2_0-200m     (lme, year) float64 ...
    Phi_2_200-600m   (lme, year) float64 ...
    ...               ...
    Phi_10_0-200m    (lme, year) float64 ...
    Phi_10_200-600m  (lme, year) float64 ...
    Phi_11_0-200m    (lme, year) float64 ...
    Phi_11_200-600m  (lme, year) float64 ...
    Phi_12_0-200m    (lme, year) float64 ...
    Phi_12_200-600m  (lme, year) float64 ...

## DPLE forecast

In [4]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{datadir}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly_to_yearly.nc'
filename = f'{datadir}/DPLE_driftcorrected_Phi_Eo_space_LME_ens_mean_monthly_to_yearly.nc'
ds_fore = xr.open_dataset(filename)
ds_fore

<xarray.Dataset>
Dimensions:          (lme: 11, Y: 64, year: 10)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y                (Y) int64 1954 1955 1956 1957 1958 ... 2014 2015 2016 2017
  * year             (year) int64 0 1 2 3 4 5 6 7 8 9
Data variables: (12/26)
    Phi_0_0-200m     (lme, Y, year) float64 ...
    Phi_0_200-600m   (lme, Y, year) float64 ...
    Phi_1_0-200m     (lme, Y, year) float64 ...
    Phi_1_200-600m   (lme, Y, year) float64 ...
    Phi_2_0-200m     (lme, Y, year) float64 ...
    Phi_2_200-600m   (lme, Y, year) float64 ...
    ...               ...
    Phi_10_0-200m    (lme, Y, year) float64 ...
    Phi_10_200-600m  (lme, Y, year) float64 ...
    Phi_11_0-200m    (lme, Y, year) float64 ...
    Phi_11_200-600m  (lme, Y, year) float64 ...
    Phi_12_0-200m    (lme, Y, year) float64 ...
    Phi_12_200-600m  (lme, Y, year) float64 ...

## Calculate Corr/Rcri Matrix for Lead Year v. LME box#

In [5]:
from sklearn.linear_model import LogisticRegression


def BrS_lowertercile(var_obs, var_test):
    # generate obs probability
    # lower tercile - 1
    # otherwise - 0
    N = len(var_obs)
    var_obs2 = np.sort(var_obs)
    bii = np.zeros(N)
    if var_obs2[-1] >= var_obs2[0]:
        bii[var_obs <= var_obs2[int(round(N/3.)-1)]] = 1
    else:
        bii[var_obs <= var_obs2[int(round(N/3.*2)-1)]] = 1
    ## model
    model = LogisticRegression()
    XX = np.zeros([N, 1])
    XX[:, 0] = var_obs.copy()
    model.fit(XX, bii)
    XXn = np.zeros([N, 1])
    XXn[:, 0] = var_test.copy()
    fii = model.predict_proba(XXn)
    #
    fii = fii[:, 1]
    # Brier Score
    BrS = np.nanmean(np.subtract(fii, bii)**2)
    return BrS


def BrS_uppertercile(var_obs, var_test):
    # generate obs probability
    # higher/upper tercile - 1
    # otherwise - 0
    N = len(var_obs)
    var_obs2 = np.sort(var_obs)
    bii = np.zeros(N)
    if var_obs2[-1] >= var_obs2[0]:
        bii[var_obs >= var_obs2[int(round(N/3.*2)-1)]] = 1
    else:
        bii[var_obs >= var_obs2[int(round(N/3.)-1)]] = 1
    ## model
    model = LogisticRegression()
    XX = np.zeros([N, 1])
    XX[:, 0] = var_obs.copy()
    model.fit(XX, bii)
    XXn = np.zeros([N, 1])
    XXn[:, 0] = var_test.copy()
    fii = model.predict_proba(XXn)
    #
    fii = fii[:, 1]
    # Brier Score
    BrS = np.nanmean(np.subtract(fii, bii)**2)
    return BrS

In [11]:
var = 'Phi'
#layer = '0-200m'
layer = '200-600m'

In [12]:

import tools
alpha = 0.05
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
# lead year range: 1,2,...,9,10
ldyrs = range(1, 11)
# persistence ACC: var1; var2
# DPLE ACC:        var2; var3
lme = [1,2,3,4,5,6,7,8,9,10,65]

lmen = ['EBS','GoA','CC','GoC','GoM','SEUS','NEUS','SS','LN','IPH','AI']
# 9/13 phis
#nphi = 9
nphi = 13
#
corr_p = np.ma.zeros([nphi, len(lme), len(ldyrs)]); corr_p.mask=True
rcri_p = np.ma.zeros([nphi, len(lme), len(ldyrs)]); rcri_p.mask=True
corr_d = corr_p.copy()
rcri_d = rcri_p.copy()
corr13 = corr_p.copy()
rcri13 = rcri_p.copy()
rmse_p = corr_p.copy()
rmse_d = corr_p.copy()
nmae_p = corr_p.copy()
nmae_d = corr_p.copy()
brs_lp = corr_p.copy()
brs_up = corr_p.copy()
brs_ld = corr_p.copy()
brs_ud = corr_p.copy()
#
for cc in range(nphi):
    for nn in range(len(lme)):
        box = lme[nn]
        var1 = ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1948:yr2-1948+1].values
        var1c = np.nanmean(ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[1954-1948:2018-1948].values)
        var1 = var1 - var1c
        #
        for lead in range(1, 11):
            var2 = ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1948+lead:yr2-1948+1+lead].values
            var2 = var2 - var1c
            var3 = ds_fore[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1954:yr2-1954+1, lead-1].values
            var3c = np.nanmean(ds_fore[f'{var}_{cc}_{layer}'].sel(lme=box)[1954-1954:2018-1954, lead-1].values)
            var3 = var3 - var3c
            # mask nan values
            mask1 = np.isnan(var1)
            mask2 = np.isnan(var2)
            mask3 = np.isnan(var3)
            mem1 = np.ma.array(var1, mask=mask1)
            mem2 = np.ma.array(var2, mask=mask2)
            mem3 = np.ma.array(var3, mask=mask3)
            #
            #Ys = mem2.copy()
            #X1s = mem1.copy()
            #X1s, Ys = (list(t) for t in zip(*sorted(zip(X1s, Ys))))
            #xs = sm.add_constant(X1s)
            #Res = sm.OLS(Ys, xs).fit()
            #pls = Res.params
            #mem11 = pls[0] + pls[1] * mem1
            #
            fmask = np.logical_and(mem1.mask, mem2.mask)
            #
            if np.sum(fmask) != len(fmask):
                nen1 = mem1[~fmask]
                nen2 = mem2[~fmask]
                corr_p[cc, nn, lead-1] = pearsonr(nen1, nen2)[0]
                neff = tools.neff3(nen1, nen2, len(nen1))
                rcri_p[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
                #
                rmse_p[cc, nn, lead-1] = np.sqrt(np.nanmean((nen1-nen2)**2))
                nmae_p[cc, nn, lead-1] = np.nanmean(np.abs(nen1-nen2)/np.nanstd(nen2))
                brs_lp[cc, nn, lead-1] = BrS_lowertercile(nen2, nen1)
                brs_up[cc, nn, lead-1] = BrS_uppertercile(nen2, nen1)
            #
            fmask = np.logical_and(mem2.mask, mem3.mask)
            #
            if np.sum(fmask) != len(fmask):
                nen2 = mem2[~fmask]
                nen3 = mem3[~fmask]
                corr_d[cc, nn, lead-1] = pearsonr(nen2, nen3)[0]
                neff = tools.neff3(nen2, nen3, len(nen2))
                rcri_d[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
                #
                rmse_d[cc, nn, lead-1] = np.sqrt(np.nanmean((nen2-nen3)**2))
                nmae_d[cc, nn, lead-1] = np.nanmean(np.abs(nen3-nen2)/np.nanstd(nen2))
                brs_ld[cc, nn, lead-1] = BrS_lowertercile(nen2, nen3)
                brs_ud[cc, nn, lead-1] = BrS_uppertercile(nen2, nen3)
            #
            fmask = np.logical_and(mem1.mask, mem3.mask)
            #
            if np.sum(fmask) != len(fmask):
                nen1 = mem1[~fmask]
                nen3 = mem3[~fmask]
                corr13[cc, nn, lead-1] = pearsonr(nen1, nen3)[0]
                neff = tools.neff3(nen1, nen3, len(nen1))
                rcri13[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
        #

In [13]:
data = xr.Dataset({"corr_p": (("nphi","nbox","nlead"),corr_p),\
                   "rcri_p": (("nphi","nbox","nlead"),rcri_p),\
                   "corr_d": (("nphi","nbox","nlead"),corr_d),\
                   "rcri_d": (("nphi","nbox","nlead"),rcri_d),\
                   "corr13": (("nphi","nbox","nlead"),corr13),\
                   "rcri13": (("nphi","nbox","nlead"),rcri13),\
                   "rmse_p": (("nphi","nbox","nlead"),rmse_p),\
                   "rmse_d": (("nphi","nbox","nlead"),rmse_d),\
                   "nmae_p": (("nphi","nbox","nlead"),nmae_p),\
                   "nmae_d": (("nphi","nbox","nlead"),nmae_d),\
                   "brs_lp": (("nphi","nbox","nlead"),brs_lp),\
                   "brs_up": (("nphi","nbox","nlead"),brs_up),\
                   "brs_ld": (("nphi","nbox","nlead"),brs_ld),\
                   "brs_ud": (("nphi","nbox","nlead"),brs_ud)})
#data['corr'] = corr1
#data['rcri'] = rcri1

In [14]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:  (nphi: 13, nbox: 11, nlead: 10)
Dimensions without coordinates: nphi, nbox, nlead
Data variables: (12/14)
    corr_p   (nphi, nbox, nlead) float64 0.8683 0.6612 ... -0.09368 -0.05975
    rcri_p   (nphi, nbox, nlead) float64 0.7067 0.7067 0.7067 ... 0.8783 0.8783
    corr_d   (nphi, nbox, nlead) float64 0.8429 0.8149 0.7699 ... 0.5574 0.502
    rcri_d   (nphi, nbox, nlead) float64 0.6664 0.7067 0.7545 ... 0.6664 0.6319
    corr13   (nphi, nbox, nlead) float64 0.8294 0.745 0.648 ... -0.07394 -0.1293
    rcri13   (nphi, nbox, nlead) float64 0.6664 0.7545 0.7545 ... 0.7067 0.6319
    ...       ...
    nmae_p   (nphi, nbox, nlead) float64 0.4313 0.6691 0.847 ... 1.222 1.183
    nmae_d   (nphi, nbox, nlead) float64 0.4707 0.4704 0.5054 ... 0.6745 0.6811
    brs_lp   (nphi, nbox, nlead) float64 0.222 0.2221 0.2221 ... 0.2239 0.2231
    brs_up   (nphi, nbox, nlead) float64 0.2279 0.2279 0.2279 ... 0.2291 0.2263
    brs_ld   (nphi, nbox, nlead) float64 0.2221 0.2221 0.2221 ... 0.2188 0.2196
    brs_ud   (nphi, nbox, nlead) float64 0.2279 0.2279 0.228 ... 0.2248 0.2256

In [15]:
%%time
#data.to_netcdf(f'{dout}/DPLE_LME_box01-11_ACC_RMSE_{var}_{layer}_leadyear_1-10.nc', mode='w')
data.to_netcdf(f'{dout}/DPLE_LME_box01-11_ACC_RMSE_{var}_Eo_space_{layer}_leadyear_1-10.nc', mode='w')

CPU times: user 16.6 ms, sys: 21.8 ms, total: 38.4 ms
Wall time: 47 ms
